In [65]:
import pandas as pd
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from matplotlib import pyplot
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix,classification_report


In [66]:
url = ('/Users/menusha/Desktop/TestPY/Anemia_NEW_edit_2.csv')
anemia_dataset = pd.read_csv(url, header = 0)
anemia_dataset.describe()


,RBC,HB,MCV,MCH,MCHC,RDW
count,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000
mean,4.915347,12.969248,80.809956,25.701696,30.712094,14.956490
std,1.259824,1.864425,10.446825,2.791919,2.714393,1.748297
min,3.000000,7.400000,58.100000,18.500000,17.000000,9.000000
25%,3.880000,12.200000,73.175000,24.100000,28.900000,13.900000
50%,4.600000,12.700000,79.850000,25.600000,30.900000,15.300000
75%,6.002500,13.700000,89.600000,27.900000,32.900000,16.300000
max,7.700000,17.200000,100.000000,30.900000,35.100000,18.600000


In [69]:
array = anemia_dataset.values
X = array[:,0:6]
Y = array[:,6]
seed = 4
validation_size = 0.30
# X_train and Y_train for preparing models
X_train,X_validate,Y_train,Y_validate = model_selection.train_test_split(X, Y, test_size=validation_size)
# split the dataset into 10 parts, train on 9 and test on 1 and repeat for all combinations
kfold = model_selection.KFold(n_splits=10, random_state=seed)
# estimate the skill of the model

# creating the base models
classifiers = []
# Part of the variance of all features is added to variances for calculation stability
model1 = GaussianNB(var_smoothing=1e-09)
classifiers.append(('nb', model1))
# lbfgs function used for multiclass classifier optimization
model2 = LogisticRegression(solver='lbfgs', multi_class='multinomial')
classifiers.append(('lr', model2))
model3 = SVC()
classifiers.append(('svm', model3))

# creating the ensemble model
ensembleModel = VotingClassifier(classifiers)
ensembleModel.fit(X_train, Y_train)
pred_rfc = ensembleModel.predict(X_validate)
# print(confusion_matrix(Y_validate,pred_rfc))
# print(classification_report(Y_validate,pred_rfc))

results = model_selection.cross_val_score(ensembleModel, X, Y, cv=kfold)
# print(results.mean())


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in v

In [7]:
import pickle
pickle.dump(ensembleModel, open('anemiaTrainedsss.pkl','wb'))